In [73]:
import pandas as pd

In [74]:
df = pd.read_csv('data/fact_sales.csv')

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customer_id       800 non-null    int64 
 1   purchase_date     800 non-null    object
 2   product_category  800 non-null    object
 3   amount            800 non-null    object
 4   transaction_id    800 non-null    int64 
 5   full_name         800 non-null    object
 6   email             800 non-null    object
 7   phone             800 non-null    object
 8   address           800 non-null    object
 9   signup_date       800 non-null    object
 10  name              800 non-null    object
 11  gender            800 non-null    object
 12  age               800 non-null    object
dtypes: int64(2), object(11)
memory usage: 81.4+ KB


In [76]:
# Total de filas
total = len(df)

# Tabla de frecuencias
customer_freq = df["transaction_id"].value_counts().reset_index()
customer_freq.columns = ["transaction_id", "count"]
customer_freq["percentage"] = (customer_freq["count"] / total) * 100

# Filtrar los que están duplicados (más de 1 aparición)
duplicates = customer_freq[customer_freq["count"] > 1]

print(duplicates)

Empty DataFrame
Columns: [transaction_id, count, percentage]
Index: []


In [77]:
# Crear una copia numérica temporal de la edad
age_num = pd.to_numeric(df["age"], errors="coerce")

# Filtrar los registros que no cumplen el rango
fuera_rango = df[
    (age_num.notna()) &  # solo filas con valor numérico
    ((age_num < 13) | (age_num > 120))
]

print("Registros con edades inválidas después de limpieza:")
print(fuera_rango)

Registros con edades inválidas después de limpieza:
Empty DataFrame
Columns: [customer_id, purchase_date, product_category, amount, transaction_id, full_name, email, phone, address, signup_date, name, gender, age]
Index: []


In [78]:
# Buscar registros con edad = 150
edades_150 = df[df["age"] == "150"]

print("Registros con edad = 150:")
print(edades_150)

Registros con edad = 150:
Empty DataFrame
Columns: [customer_id, purchase_date, product_category, amount, transaction_id, full_name, email, phone, address, signup_date, name, gender, age]
Index: []


In [79]:
# --- Filas con emails duplicados (excluyendo "No especificado") ---
rows_email_dupes = df[
    (df["email"].notna()) &
    (df["email"] != "No especificado") &
    (df.duplicated(subset=["email"], keep=False))
]

print("=== Filas con emails duplicados en Fact Sales (sin 'No especificado') ===")
print(rows_email_dupes.sort_values("email").head(50))  # primeras 50 filas

# --- Filas con phones duplicados (excluyendo "No especificado") ---
rows_phone_dupes = df[
    (df["phone"].notna()) &
    (df["phone"] != "No especificado") &
    (df.duplicated(subset=["phone"], keep=False))
]

print("\n=== Filas con phones duplicados en Fact Sales (sin 'No especificado') ===")
print(rows_phone_dupes.sort_values("phone").head(50))  # primeras 50 filas

=== Filas con emails duplicados en Fact Sales (sin 'No especificado') ===
Empty DataFrame
Columns: [customer_id, purchase_date, product_category, amount, transaction_id, full_name, email, phone, address, signup_date, name, gender, age]
Index: []

=== Filas con phones duplicados en Fact Sales (sin 'No especificado') ===
Empty DataFrame
Columns: [customer_id, purchase_date, product_category, amount, transaction_id, full_name, email, phone, address, signup_date, name, gender, age]
Index: []


In [80]:
print(rows_phone_dupes.sort_values("phone").head(5)) 

Empty DataFrame
Columns: [customer_id, purchase_date, product_category, amount, transaction_id, full_name, email, phone, address, signup_date, name, gender, age]
Index: []


In [81]:
# Agrupar por claves de unicidad (sin amount)
duplicados_amount = (
    df.groupby(["customer_id", "purchase_date", "product_category", "full_name"])
    ["amount"]
    .nunique()
    .reset_index()
)

# Filtrar los grupos con más de 1 valor único de amount
diferentes_amount = duplicados_amount[duplicados_amount["amount"] > 1]

print("Registros con mismo cliente/transacción pero diferente amount:")
print(diferentes_amount)


Registros con mismo cliente/transacción pero diferente amount:
Empty DataFrame
Columns: [customer_id, purchase_date, product_category, full_name, amount]
Index: []


In [82]:
ids_conflictivos = diferentes_amount[
    ["customer_id", "purchase_date", "product_category", "full_name"]
]

conflictos = df.merge(ids_conflictivos, on=["customer_id", "purchase_date", "product_category", "full_name"])

print(conflictos)


Empty DataFrame
Columns: [customer_id, purchase_date, product_category, amount, transaction_id, full_name, email, phone, address, signup_date, name, gender, age]
Index: []
